In [1]:
import pandas as pd
import glob, os

### Merging 12 months of sales data into a single file

Also a single csv just in case that we need it

In [94]:
csv_files = [file for files in os.listdir("../datasets/Sales_Data")]

# We also can use:
#csv_files = glob.glob(os.path.join(mainpath, "*.csv"))

data = pd.DataFrame()

for file in csv_files:
    new_data =  pd.read_csv(file)
    data = pd.concat([data,new_data],axis=0)
    
# remove headers
trash_data = data.loc[data['Order Date']=='Order Date'].index
data.drop(trash_data, inplace = True) 

# save all data in a single csv file
data.to_csv(mainpath+"/full_dataset/Sales_all_2019.csv",index=False)


In [ ]:
""" 
First solution (before realizing that I had to clean the data):

data['DateTime'] = pd.to_datetime(data['Order Date'], format='%m/%d/%y %H:%M', errors="coerce")
#data['Quantity Ordered'] =  data['Quantity Ordered'].astype(int)
#data['Price Each'] =  data['Price Each'].astype(float)

grouped_by_month = data.groupby(data["DateTime"].dt.month)
groups_keys = grouped_by_month.groups.keys()

sales_for_each_month = {}
for group in groups_keys:
    sales_for_each_month[group] = 0
    month_sales = grouped_by_month.get_group(group)
    for index, row in month_sales.iterrows():
        try:
            value = float(row['Price Each']) * int(row['Quantity Ordered'])
        except TypeError:
            indices_rotos.append(index)
            continue
        sales_for_each_month[group] += value
sales_for_each_month
"""

### Clean data

In [111]:
data = pd.read_csv(mainpath+"/full_dataset/Sales_all_2019.csv")
data = data.dropna(axis=0,how="all")

data['DateTime'] = pd.to_datetime(data['Order Date'], format='%m/%d/%y %H:%M')
                                  
data['Month'] = data['Order Date'].str[:2]
data['Month'] = data['Month'].astype('int32')

data = data.drop(['Order Date'],axis=1)

data['Order ID'] = data['Order ID'].astype(int)
data['Product'] = data['Product'].astype(str)
data['Quantity Ordered'] = data['Quantity Ordered'].astype(int)
data['Price Each'] = data['Price Each'].astype(float)
data['Purchase Address'] = data['Purchase Address'].astype(str)

data.head()

,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,DateTime,Month
0,236670,Wired Headphones,2,11.99,"359 Spruce St, Seattle, WA 98101",2019-08-31 22:21:00,8
1,236671,Bose SoundSport Headphones,1,99.99,"492 Ridge St, Dallas, TX 75001",2019-08-15 15:11:00,8
2,236672,iPhone,1,700.00,"149 7th St, Portland, OR 97035",2019-08-06 14:40:00,8
3,236673,AA Batteries (4-pack),2,3.84,"631 2nd St, Los Angeles, CA 90001",2019-08-29 20:59:00,8
4,236674,AA Batteries (4-pack),2,3.84,"736 14th St, New York City, NY 10001",2019-08-15 19:53:00,8


In [114]:
data.shape

(155441, 7)

### Which was the best month for sales?